# Parameters

In [1]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot for this month must be available.
METRICS_MONTH_TEXT = "2019-02"
MEDIAWIKI_HISTORY_SNAPSHOT = "2019-02"

# Imports

In [2]:
import datetime as dt
from functools import reduce
import io
from pathlib import Path
import re
import time

from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import wmfdata as wmf
from wmfdata import hive
from wmfdata.utils import mediawiki_dt, print_err, pd_display_all

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    "metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics

In [ ]:
queries = {
    "active_editors": {
        "file": "queries/active_editors.sql",
        "engine": "hive"
    },
#     "mobile_edits": {
#         "file": "queries/mobile_edits.sql",
#         "engine": "hive"
#     },
    "edits": {
        "file": "queries/edits.hql",
        "engine": "hive"
    },
    "new_editor_retention": {
        "file": "queries/new_editor_retention.hql",
         "engine": "hive"
    },
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_edits_editors": {
        "file": "queries/mobile-heavy_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_new_editor_retention": {
        "file": "queries/mobile-heavy_new_editor_retention.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running edits on hive...


# Content metrics via API

In [ ]:
NEW_PAGES_API = (
    "https://wikimedia.org/api/rest_v1/metrics/edited-pages/new/" +
    "{project}/all-editor-types/{page_type}/monthly/{start}/{end}"
)

headers = {
    "User-Agent": "https://github.com/wikimedia-research/Editing-movement-metrics (bot)"
}

# Create container for results
api_results = []

def get_new_pages(
    project="all-projects",
    page_type="content",
    start=date_params["api_metrics_month_first_day"],
    end=date_params["api_metrics_month_day_after"]
):
    url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )
    
    r = requests.get(url, headers=headers)
    data = r.json()["items"][0]["results"]
    frame = pd.DataFrame(data)
    frame["timestamp"] = pd.to_datetime(frame["timestamp"])
    frame = frame.rename(columns={"timestamp": "month"})
    
    return frame

## Total

In [ ]:
total_new = get_new_pages().rename(columns={"new_pages": "net_new_content_pages"})
api_results.append(total_new)

## Wikidata

In [ ]:
new_wd = get_new_pages(
    project="wikidata.org"
).rename(columns={
    "new_pages": "net_new_Wikidata_entities"
})
api_results.append(new_wd)

## Commons

In [ ]:
new_commons = get_new_pages(
    project="commons.wikimedia.org"
).rename(columns={
    "new_pages": "net_new_Commons_content_pages"
})
api_results.append(new_commons)

## Wikipedias

In [ ]:
# Get a list of project URLs (each one in a 1-tuple)
wp_domains = hive.run("""
select domain_name
from canonical_data.wikis
where database_group = "wikipedia"
""", fmt="raw")

# Query the API for each project and append records to a list
results = []
n = len(wp_domains)

for idx, val in enumerate(wp_domains):
    domain = val[0]
    
    if idx % 10 == 0:
        msg = "Now on the {}th project of {} ({})"
        print_err(msg.format(idx, n, domain))
        
    frame = get_new_pages(project=domain).reset_index()
    frame["project"] = domain
    records = frame.to_dict("records")
    results.extend(records)
    
    # Sleep 20 milliseconds
    time.sleep(0.02)

# Turn the big list of records into a data frame
new_per_wp = pd.DataFrame(results)

# Sum across projects to get new Wikipedia articles per month
new_wp = new_per_wp.groupby("month").agg(
    {"new_pages": "sum"}
).rename(columns={"new_pages": "net_new_Wikipedia_articles"}).reset_index()

api_results.append(new_wp)

In [ ]:
# Strip timezones returned by API so our month columns merge nicely
for df in api_results:
    df["month"] = df["month"].dt.tz_localize(None)

# Combining and saving metrics

In [14]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]
results.extend(api_results)

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,active_editors,data_edits,existing_active_editors,global_south_active_editors,global_south_edits,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,mobile_edits,net_new_Commons_content_pages,net_new_Wikidata_entities,net_new_Wikipedia_articles,net_new_content_pages,new_active_editors,new_editor_retention,nonbot_nondata_nonupload_edits,revert_rate,second_month_active_editors,total_edits,uploads
month,,,,,,,,,,,,,,,,,,,,,,
2018-10-01,82567.0,23090068.0,60815.0,19891.0,1979771.0,1979769.0,3825.0,957794.0,0.050485,433507.0,1231434.0,549132.0,1147804.0,191087.0,2101982.0,16690.0,0.066411,13382193.0,0.073543,4251.0,43471807.0,588857.0
2018-11-01,82584.0,23908469.0,61330.0,20008.0,2015177.0,2015175.0,3639.0,1150793.0,0.043329,456985.0,1228780.0,549451.0,1272149.0,192340.0,2268787.0,15666.0,0.068742,13563561.0,0.071465,4735.0,44349706.0,592679.0
2018-12-01,81075.0,21426310.0,61161.0,20359.0,2067577.0,2067577.0,4046.0,1125381.0,0.047045,488735.0,1260692.0,512338.0,947571.0,186233.0,2128146.0,14753.0,0.072074,13490420.0,0.065287,4297.0,41966740.0,551296.0
2019-01-01,86776.0,23223444.0,65161.0,21367.0,2434941.0,2434941.0,4228.0,988233.0,0.043358,512674.0,1416465.0,542425.0,951976.0,200366.0,1938938.0,17108.0,0.054232,14917046.0,0.060343,3652.0,46294961.0,587164.0
2019-02-01,82554.0,22674118.0,61981.0,19743.0,2241877.0,2241875.0,3901.0,966696.0,0.045479,438437.0,NaN,494725.0,574002.0,195100.0,1455387.0,16335.0,0.054653,13415954.0,0.060140,4238.0,44131944.0,534391.0


In [15]:
metrics.to_csv(FILENAME, sep="\t")